##### Capstone Project: With student academic data from the Fall of 2019 and Fall 2020, can we predict undergaduate student outcomes for students within the College of Natural Sciences at UT Austin? Can we make predictions about student’s grades based upon major, program, instructor, or classification? For this data set, grade is the dependent variable.  

##### The courses I am most interested in comparing are six first-year classes and two second-year classes (CH 301, BIO 311C, M305G, M408N, M408C, M408R, CH 320M, BIO 325)


In [1]:
# Importing all of the necessary basics and installed pandas_profiling seperately -DS Basics
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

#SKLearn Stuff
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#helpers
%matplotlib inline

In [2]:
#The following data was separated into 12 different csv files inititally. 
#I utilized my computer terminal to combine the documents successfully. 
#This data will need to go through extensive cleaning but is formatted correctly
#Received Error tokenizing data meaning the file needs to be cleaned extensively. I assume it's the commas it has?
#Renamed all the columns as well 
data = pd.read_csv('/Users/ma25756/Desktop/combine/combined.csv',header=None,error_bad_lines=False,skiprows=1)
data.columns = ["FTIC","Major",'Major 2',"CCYYS","Course","Unique","Title","Grade","Instructor","Advisor","Has > 1"]

b'Skipping line 6592: expected 11 fields, saw 21\nSkipping line 30080: expected 11 fields, saw 21\nSkipping line 42773: expected 11 fields, saw 21\nSkipping line 47137: expected 11 fields, saw 21\nSkipping line 48799: expected 11 fields, saw 21\nSkipping line 52417: expected 11 fields, saw 21\nSkipping line 58387: expected 11 fields, saw 21\n'
b'Skipping line 89081: expected 11 fields, saw 21\nSkipping line 97818: expected 11 fields, saw 21\nSkipping line 100158: expected 11 fields, saw 21\nSkipping line 102959: expected 11 fields, saw 21\n'


In [4]:
#Want to take a look at the data. Right off the bat the Major 1 needs to be cleaned up extensively.
data.head()

,FTIC,Major,Major 2,CCYYS,Course,Unique,Title,Grade,Instructor,Advisor,Has > 1
0,0,E \t\t \t\tBIO/ \t\t\t\t\t\t\tBIO \...,NaN,20199,BIO 359K,48120,Principles Of Animal Behavior,A-,"Ryan, Michael \t ryanmj",NaN,NaN
1,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,NaN,20199,C S 378,50585,Ethical Hacking,A-,"Prosise, Chris \t cp34499","Guzman, Peter (pguzman)",NaN
2,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,NaN,20199,C S 371L,50515,Ios Mobile Computing,A-,"Bulko, William \t bulkowc","Guzman, Peter (pguzman)",This occurrence was > 1 of these Grades
3,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,ECO 441K,33775,Introduction To Econometrics,A-,"Donald, Stephen \t donalds","Roberts, Douglas (dmr99)",NaN
4,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,GOV 310L,37420,American Government,A-,"O'Brien, Shannon \t sbo69","Roberts, Douglas (dmr99)",This occurrence was > 1 of these Grades


In [5]:
data.shape

(105654, 11)

In [6]:
data.describe()

,FTIC,CCYYS,Unique
count,105654.000000,105654.000000,105654.000000
mean,18311.281163,20204.039468,44179.216168
std,5862.939629,4.999868,14081.644880
min,0.000000,20199.000000,260.000000
25%,20179.000000,20199.000000,41304.000000
50%,20189.000000,20209.000000,49134.000000
75%,20199.000000,20209.000000,52645.000000
max,20209.000000,20209.000000,64870.000000


## Data Cleansing

In [27]:
#replacing NAN with a static value (No Advisor)
data["Advisor"].fillna("No Advisor", inplace=True)

In [28]:
data.head()

,FTIC,Major,CCYYS,Course,Unique,Title,Grade,Instructor,Advisor
0,0,E \t\t \t\tBIO/ \t\t\t\t\t\t\tBIO \...,20199,BIO 359K,48120,Principles Of Animal Behavior,A-,"Ryan, Michael \t ryanmj",No Advisor
1,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 378,50585,Ethical Hacking,A-,"Prosise, Chris \t cp34499","Guzman, Peter (pguzman)"
2,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 371L,50515,Ios Mobile Computing,A-,"Bulko, William \t bulkowc","Guzman, Peter (pguzman)"
3,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,ECO 441K,33775,Introduction To Econometrics,A-,"Donald, Stephen \t donalds","Roberts, Douglas (dmr99)"
4,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,GOV 310L,37420,American Government,A-,"O'Brien, Shannon \t sbo69","Roberts, Douglas (dmr99)"


In [29]:
df = data

In [30]:
df.head()

,FTIC,Major,CCYYS,Course,Unique,Title,Grade,Instructor,Advisor
0,0,E \t\t \t\tBIO/ \t\t\t\t\t\t\tBIO \...,20199,BIO 359K,48120,Principles Of Animal Behavior,A-,"Ryan, Michael \t ryanmj",No Advisor
1,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 378,50585,Ethical Hacking,A-,"Prosise, Chris \t cp34499","Guzman, Peter (pguzman)"
2,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 371L,50515,Ios Mobile Computing,A-,"Bulko, William \t bulkowc","Guzman, Peter (pguzman)"
3,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,ECO 441K,33775,Introduction To Econometrics,A-,"Donald, Stephen \t donalds","Roberts, Douglas (dmr99)"
4,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,GOV 310L,37420,American Government,A-,"O'Brien, Shannon \t sbo69","Roberts, Douglas (dmr99)"


In [35]:
#Removing information between the parenthesis for advisor
df["Advisor"] = df ["Advisor"].str.replace(r"\(.*?\)", "")

In [36]:
#Confirm drop of all info between parenthesis within Advisor column
df.head()

,FTIC,Major,CCYYS,Course,Unique,Title,Grade,Instructor,Advisor
0,0,E \t\t \t\tBIO/ \t\t\t\t\t\t\tBIO \...,20199,BIO 359K,48120,Principles Of Animal Behavior,A-,"Ryan, Michael \t ryanmj",No Advisor
1,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 378,50585,Ethical Hacking,A-,"Prosise, Chris \t cp34499","Guzman, Peter"
2,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 371L,50515,Ios Mobile Computing,A-,"Bulko, William \t bulkowc","Guzman, Peter"
3,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,ECO 441K,33775,Introduction To Econometrics,A-,"Donald, Stephen \t donalds","Roberts, Douglas"
4,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,GOV 310L,37420,American Government,A-,"O'Brien, Shannon \t sbo69","Roberts, Douglas"


In [44]:
#Removing eid information after instructor name 
df["Instructor"] = df["Instructor"].str.replace("(\t).*","")

In [45]:
#Confirm replacement of the eids after instructor name
#Successful replacement
df.head()

,FTIC,Major,CCYYS,Course,Unique,Title,Grade,Instructor,Advisor
0,0,E \t\t \t\tBIO/ \t\t\t\t\t\t\tBIO \...,20199,BIO 359K,48120,Principles Of Animal Behavior,A-,"Ryan, Michael",No Advisor
1,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 378,50585,Ethical Hacking,A-,"Prosise, Chris","Guzman, Peter"
2,20169,E \t\t \t\tC_S/ \t\t\t\t\t\t\tC_S \...,20199,C S 371L,50515,Ios Mobile Computing,A-,"Bulko, William","Guzman, Peter"
3,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,ECO 441K,33775,Introduction To Econometrics,A-,"Donald, Stephen","Roberts, Douglas"
4,20179,L \t\t \t\tECO/ \t\t\t\t\t\t\tECO \...,20199,GOV 310L,37420,American Government,A-,"O'Brien, Shannon","Roberts, Douglas"


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105654 entries, 0 to 105653
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   FTIC        105654 non-null  int64 
 1   Major       105654 non-null  object
 2   CCYYS       105654 non-null  int64 
 3   Course      105654 non-null  object
 4   Unique      105654 non-null  int64 
 5   Title       105654 non-null  object
 6   Grade       105654 non-null  object
 7   Instructor  105654 non-null  object
 8   Advisor     105654 non-null  object
dtypes: int64(3), object(6)
memory usage: 7.3+ MB


In [52]:
df.describe()

,FTIC,CCYYS,Unique
count,105654.000000,105654.000000,105654.000000
mean,18311.281163,20204.039468,44179.216168
std,5862.939629,4.999868,14081.644880
min,0.000000,20199.000000,260.000000
25%,20179.000000,20199.000000,41304.000000
50%,20189.000000,20209.000000,49134.000000
75%,20199.000000,20209.000000,52645.000000
max,20209.000000,20209.000000,64870.000000


In [53]:
#Unclear if I should eliminate the duplicates or keep them. Will do more digging and research about this.
df.duplicated ().sum ()

13695

#### Action: Determine how to clean up the major column in the data. 